In [1]:
from coffea.nanoaod import NanoEvents
from uproot_methods import TVector2Array, TLorentzVectorArray

fname = 'root://cmseos.fnal.gov//store/group/lpccoffea/coffeabeans/NanoAODv6/nano_2018/MET/NanoTuples-2018_Run2018A-17Sep2018-v1/191206_204012/0000/nano_1.root'
events = NanoEvents.from_file(fname)

In [4]:
import uproot
file = uproot.open(fname)

In [6]:
file.keys()

[b'eej;1', b'evj;1', b'vvj;1', b'aj;1']

In [12]:
file['eej'].show()

                    0                                                     1.2317
                    +----------------------------------------------------------+
[-inf, 30)   0      |                                                          |
[30, 40)     1.0604 |**************************************************        |
[40, 50)     1.0539 |**************************************************        |
[50, 60)     1.0766 |***************************************************       |
[60, 70)     1.0642 |**************************************************        |
[70, 80)     1.0662 |**************************************************        |
[80, 90)     1.0637 |**************************************************        |
[90, 100)    1.0759 |***************************************************       |
[100, 110)   1.0742 |***************************************************       |
[110, 120)   1.0768 |***************************************************       |
[120, 130)   1.0837 |*******

In [2]:
j = events.Jet
met = events.MET

j['p4'] = TLorentzVectorArray.from_ptetaphim(j.pt, j.eta, j.phi, j.mass)
j['T'] = TVector2Array.from_polar(j.pt, j.phi)
leading_j = j[j.pt.argmax()]

met['T'] = TVector2Array.from_polar(met.pt, met.phi)

In [3]:
len((met['T'].delta_phi(leading_j['T']).sum()) > 1.5)

49523

In [5]:
len(abs((met['T'].delta_phi(leading_j['T'])).sum()) > 1.5)

49523

In [21]:
x&y

array([1, 1, 1, ..., 1, 1, 1])

In [23]:
e = events.Electron
mu = events.Muon
e['p4'] = TLorentzVectorArray.from_ptetaphim(e.pt, e.eta, e.phi, e.mass)
mu['p4'] = TLorentzVectorArray.from_ptetaphim(mu.pt, mu.eta, mu.phi, mu.mass)
LJ_Ele = leading_j['p4'].cross(e['p4'])
DeltaR_LJ_Ele = LJ_Ele.i0.delta_r(LJ_Ele.i1)
DeltaR_LJ_Ele_mask = (DeltaR_LJ_Ele < 3.4).any()

LJ_Mu = leading_j['p4'].cross(mu['p4'])
DeltaR_LJ_Mu = LJ_Mu.i0.delta_r(LJ_Mu.i1)
DeltaR_LJ_Mu_mask = (DeltaR_LJ_Mu < 3.4).any()

In [42]:
((DeltaR_LJ_Mu < 3.4).sum()).dtype

dtype('int64')

In [43]:
((DeltaR_LJ_Mu < 3.4).any()).dtype

dtype('bool')

In [4]:
#!/usr/bin/env python
import lz4.frame as lz4f
import cloudpickle
import json
import pprint
import numpy as np
import awkward
np.seterr(divide='ignore', invalid='ignore', over='ignore')
from coffea.arrays import Initialize
from coffea import hist, processor
from coffea.util import load, save
from coffea.jetmet_tools import FactorizedJetCorrector, JetCorrectionUncertainty, JetTransformer, JetResolution, JetResolutionScaleFactor
from optparse import OptionParser
from uproot_methods import TVector2Array, TLorentzVectorArray

In [1]:
!pwd

/uscms_data/d3/runiyal/lep_kr/decaf/analysis/Rishabh_jupyter-NB


In [9]:
get_corrections = load('../data/corrections.coffea')

In [11]:
get_deepflav_weight = get_corrections['get_btag_weight']['deepflav']

In [16]:
get_deepflav_weight['2018']['loose']

<bound method BTagCorrector.btag_weight of <__main__.BTagCorrector object at 0x7f7f805fa978>>

In [22]:
np.minimum(1, [-1,2,3,4])

array([-1,  1,  1,  1])

In [29]:
a = awkward.fromiter([[1,2],[3]])

In [30]:
a.prod()

array([2, 3])

In [7]:
events.size

23452

In [17]:
events.GenPart.pdgId

<JaggedArray [[1 -1 23 ... -211 22 22] [-1 1 23 ... -421 21 431] [-1 21 23 ... -14 22 22] ... [-1 1 23 23 -13 13] [2 -2 23 ... -11 22 11] [1 -1 23 ... 22 22 22]] at 0x7f48c70de358>

In [104]:
# I want to select only events coming from  W [pdgId = 24]

electron = events.Electron
muons = events.Muon

In [106]:
electron.columns

['deltaEtaSC',
 'dr03EcalRecHitSumEt',
 'dr03HcalDepth1TowerSumEt',
 'dr03TkSumPt',
 'dr03TkSumPtHEEP',
 'dxy',
 'dxyErr',
 'dz',
 'dzErr',
 'eCorr',
 'eInvMinusPInv',
 'energyErr',
 'eta',
 'hoe',
 'ip3d',
 'jetPtRelv2',
 'jetRelIso',
 'mass',
 'miniPFRelIso_all',
 'miniPFRelIso_chg',
 'mvaFall17V1Iso',
 'mvaFall17V1noIso',
 'mvaFall17V2Iso',
 'mvaFall17V2noIso',
 'pfRelIso03_all',
 'pfRelIso03_chg',
 'phi',
 'pt',
 'r9',
 'sieie',
 'sip3d',
 'mvaTTH',
 'charge',
 'cutBased',
 'cutBased_Fall17_V1',
 'pdgId',
 'tightCharge',
 'vidNestedWPBitmap',
 'vidNestedWPBitmapHEEP',
 'convVeto',
 'cutBased_HEEP',
 'isPFcand',
 'lostHits',
 'mvaFall17V1Iso_WP80',
 'mvaFall17V1Iso_WP90',
 'mvaFall17V1Iso_WPL',
 'mvaFall17V1noIso_WP80',
 'mvaFall17V1noIso_WP90',
 'mvaFall17V1noIso_WPL',
 'mvaFall17V2Iso_WP80',
 'mvaFall17V2Iso_WP90',
 'mvaFall17V2Iso_WPL',
 'mvaFall17V2noIso_WP80',
 'mvaFall17V2noIso_WP90',
 'mvaFall17V2noIso_WPL',
 'seedGain',
 'genPartFlav',
 'cleanmask',
 'matched_jet',
 'matched

In [6]:
electron.matched_gen.columns

['eta',
 'mass',
 'phi',
 'pt',
 'pdgId',
 'status',
 'statusFlags',
 'parent',
 'children']

In [82]:
wemu_events = events[(electron.counts >= 1) 
      | (electron.counts >= 1)]

In [83]:
met = events.MET

In [84]:
wemu_events.columns

['FsrPhoton',
 'SoftActivityJetNjets5',
 'GenMET',
 'fixedGridRhoFastjetCentral',
 'Flag',
 'Jet',
 'luminosityBlock',
 'CaloMET',
 'Photon',
 'LHEPart',
 'genWeight',
 'GenJetAK8',
 'SoftActivityJet',
 'LHE',
 'FatJet',
 'GenPart',
 'genTtbarId',
 'SoftActivityJetHT2',
 'HLTriggerFinalPath',
 'event',
 'GenDressedLepton',
 'PuppiMET',
 'MET',
 'fixedGridRhoFastjetCentralCalo',
 'SoftActivityJetHT5',
 'Tau',
 'LHEWeight',
 'PSWeight',
 'SoftActivityJetHT',
 'btagWeight',
 'Electron',
 'LHEReweightingWeight',
 'GenJet',
 'fixedGridRhoFastjetAll',
 'HLTriggerFirstPath',
 'fixedGridRhoFastjetCentralNeutral',
 'fixedGridRhoFastjetCentralChargedPileUp',
 'SoftActivityJetNjets2',
 'Pileup',
 'SoftActivityJetNjets10',
 'HLT',
 'L1Reco',
 'Generator',
 'HTXS',
 'SoftActivityJetHT10',
 'TkMET',
 'SubJet',
 'SubGenJetAK8',
 'OtherPV',
 'LHEPdfWeight',
 'IsoTrack',
 'PV',
 'ChsMET',
 'GenVisTau',
 'Muon',
 'L1',
 'RawMET',
 'L1simulation',
 'LHEScaleWeight',
 'CorrT1METJet',
 'TrigObj',
 'run',
 

In [91]:
len(electron.pt)

40

In [92]:
wemu_events.GenPart[
    ((abs(wemu_events.GenPart.pdgId) == 11)
    | (abs(wemu_events.GenPart.pdgId) == 13))
    & wemu_events.GenPart.hasFlags(['isPrompt', 'isLastCopy'])
].distinctParent.pdgId




<JaggedArray [[23 23] [23 23] [] ... [23 23] [] [23 23]] at 0x7fc0f425e4a8>

In [93]:
events.GenPart.parent.distinctParent.pdgId

<JaggedArray [[None None None ... 23 -15 -15] [None None None ... None None None] [None None None ... -413 -1 -1] ... [None None None -1 -1 -1] [None None None ... 23 23 23] [None None None ... 15 -15 -15]] at 0x7fc0f425ef60>

In [97]:
(events.GenPart.pdgId == 23).any()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [99]:
events.Flag.columns

['ecalBadCalibFilterV2',
 'HBHENoiseFilter',
 'HBHENoiseIsoFilter',
 'CSCTightHaloFilter',
 'CSCTightHaloTrkMuUnvetoFilter',
 'CSCTightHalo2015Filter',
 'globalTightHalo2016Filter',
 'globalSuperTightHalo2016Filter',
 'HcalStripHaloFilter',
 'hcalLaserEventFilter',
 'EcalDeadCellTriggerPrimitiveFilter',
 'EcalDeadCellBoundaryEnergyFilter',
 'ecalBadCalibFilter',
 'goodVertices',
 'eeBadScFilter',
 'ecalLaserCorrFilter',
 'trkPOGFilters',
 'chargedHadronTrackResolutionFilter',
 'muonBadTrackFilter',
 'BadChargedCandidateFilter',
 'BadPFMuonFilter',
 'BadChargedCandidateSummer16Filter',
 'BadPFMuonSummer16Filter',
 'trkPOG_manystripclus53X',
 'trkPOG_toomanystripclus53X',
 'trkPOG_logErrorTooManyClusters',
 'METFilters']